<a href="https://colab.research.google.com/github/deeplearningexplore/siamese/blob/main/question_pairs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c quora-question-pairs

Saving kaggle.json to kaggle.json
100% 114M/114M [00:03<00:00, 20.8MB/s]
100% 114M/114M [00:03<00:00, 37.7MB/s]
100% 4.95M/4.95M [00:00<00:00, 27.0MB/s]

 24% 5.00M/21.2M [00:00<00:02, 6.65MB/s]
100% 21.2M/21.2M [00:00<00:00, 25.4MB/s]
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import pandas as pd

%matplotlib inline

In [3]:
zip = zipfile.ZipFile("test.csv.zip", "r")
zip.extractall("./data")


In [4]:
zip = zipfile.ZipFile("train.csv.zip", "r")
zip.extractall("./data")

zip.close()

In [5]:
!rm "train.csv.zip"
!rm "test.csv.zip"

In [6]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
print(train_data.head())
print(test_data.head())

   id  qid1  ...                                          question2 is_duplicate
0   0     1  ...  What is the step by step guide to invest in sh...            0
1   1     3  ...  What would happen if the Indian government sto...            0
2   2     5  ...  How can Internet speed be increased by hacking...            0
3   3     7  ...  Find the remainder when [math]23^{24}[/math] i...            0
4   4     9  ...            Which fish would survive in salt water?            0

[5 rows x 6 columns]
   test_id  ...                                          question2
0        0  ...  Why did Microsoft choose core m3 and not core ...
1        1  ...        How much cost does hair transplant require?
2        2  ...                      What you send money to China?
3        3  ...                                  What foods fibre?
4        4  ...                     How their can I start reading?

[5 rows x 3 columns]


In [7]:
train_data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
train_data = train_data[train_data["is_duplicate"]==1]
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [9]:
train_d1 = train_data['question1'].values
train_d2 = train_data['question2'].values

In [10]:
train_d1.shape

(149263,)

In [11]:
train_d = np.concatenate([train_d1,train_d2])

In [12]:
train_d.shape

(298526,)

In [13]:
oov_token = "<ukw>"
pad = "post"


In [19]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_d)
word_index = tokenizer.word_index


In [20]:
len(word_index)

31468

In [21]:
train_data1 = tokenizer.texts_to_sequences(train_d1)

In [22]:
train_data2 = tokenizer.texts_to_sequences(train_d2)

In [39]:
word_index["ambulance"]

19614

In [25]:
def data_generator(q1,q2,batch_size):
  indexes = list(range(len(q1)))
  indexes = tf.random.shuffle(indexes)
  length = len(q1)
  input1 = []
  input2 = []
  index = 0

  while True:
    if index==length:
      index = 0
      indexes = tf.random.shuffle(indexes)
    
    input1.append(q1[indexes[index]])
    input2.append(q2[indexes[index]])

    index+=1

    if len(input1)==batch_size:
      max_length = np.max((np.max([len(x) for x in input1]),np.max([len(x) for x in input2])))
      max_length = 2**int(np.ceil(np.log2(max_length)))

      input1 = tf.keras.preprocessing.sequence.pad_sequences(input1,maxlen=max_length,padding=pad,truncating=pad)
      input2 = tf.keras.preprocessing.sequence.pad_sequences(input2,maxlen=max_length,padding=pad,truncating=pad)

      yield np.array(input1),np.array(input2)

      input1 = []
      input2 = []



In [26]:
a,b=next(data_generator(train_data1,train_data2,1))

In [27]:
print(a[0])
print(b[0])

[   3   11   26   12    2   46 1607 5793  746   18 7184    9   30    0
    0    0]
[   3   11   26   46 7184 1607   18   10  691  813 7184  466    0    0
    0    0]


In [28]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

--2020-10-12 15:31:28--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  8.32MB/s    in 71s     

2020-10-12 15:32:40 (9.15 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [29]:
zip = zipfile.ZipFile("wiki-news-300d-1M.vec.zip", "r")
zip.extractall("./data")
zip.close()

In [30]:
!rm "wiki-news-300d-1M.vec.zip"

In [32]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
    return data

embedding_vectors =  load_vectors("data/wiki-news-300d-1M.vec")

In [34]:
len(embedding_vectors)

999994

In [36]:
vocab_size =  len(tokenizer.word_index) + 1
dimensions = 300
batch_size = 128

In [38]:
embedding_matrix = np.zeros((vocab_size, dimensions))
for word,index in word_index.items():
  val = embedding_vectors.get(word, None)
  if val is not None:
    embedding_matrix[index] = val

In [40]:
del embedding_vectors